# Advent of Code 2020-12-08

## Part 1

```
--- Day 7: Handy Haversacks ---
You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

A bright white bag, which can hold your shiny gold bag directly.
A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

```



In [55]:
from collections import deque
import math
import re

In [2]:
def parse_rule(s):
    pattern = r'''(?P<parent_color>[\w\s]+)
    \s+bags\s+contain\s+
    (?P<children>.+)
    \.
    '''
    p = re.compile(pattern, re.VERBOSE)
    m = p.match(s)
    parent_color = m.group('parent_color')
    children = m.group('children')
    child_dict = {}
    item_pattern = r'''(?P<count>\d+)\s+(?P<color>[\w\s]+)\s+bags?'''
    item_p = re.compile(item_pattern, re.VERBOSE)
    if children != 'no other bags':
        for child in children.split(','):
            child = child.strip()
            item_m = item_p.match(child)
            count = int(item_m.group('count'))
            color = item_m.group('color')
            child_dict[color] = count
    return parent_color, child_dict

parse_rule('muted tomato bags contain 1 bright brown bag, 1 dotted gold bag, 2 faded gray bags, 1 posh yellow bag.')

('muted tomato',
 {'bright brown': 1, 'dotted gold': 1, 'faded gray': 2, 'posh yellow': 1})

In [3]:
parse_rule('faded violet bags contain no other bags.')

('faded violet', {})

In [10]:
def parse_graph(s):
    graph = {}
    for line in s.split('\n'):
        line = line.strip()
        if not line:
            continue
        parent_color, child_dict = parse_rule(line)
        graph[parent_color] = child_dict
    return graph

def process_input():
    with open('1207_input.txt', 'r') as f:
        return parse_graph(f.read())
                        
graph = process_input()
graph['muted tomato']

{'bright brown': 1, 'dotted gold': 1, 'faded gray': 2, 'posh yellow': 1}

In [11]:
set(graph['muted tomato'].keys()) - set(['muted tomato'])

{'bright brown', 'dotted gold', 'faded gray', 'posh yellow'}

In [12]:
def bfs_paths(graph, source, target):
    if graph.get(source) is None:
        return None
    if graph.get(target) is None:
        return None
    queue = [(source, [source])]
    while queue:
        (vertex, path) = queue.pop(0)
        # print('vertex:', vertex)
        # print('path:', path)
        # print('nodes:', set(graph[vertex].keys()) - set(path))
        for node in set(graph[vertex].keys()) - set(path):
            if node == target:
                yield path + [node]
            else:
                queue.append((node, path + [node]))

def shortest_path(graph, source, target):
    try:
        return next(bfs_paths(graph, source, target))
    except StopIteration:
        return None

print(shortest_path(graph, 'muted tomato', 'dotted gold'))
print(shortest_path(graph, 'muted tomato', 'dotted chartreuse'))
print(shortest_path(graph, 'muted tomato', 'muted tomato'))
print(shortest_path(graph, 'shiny gold', 'shiny gold'))


['muted tomato', 'dotted gold']
None
None
None


In [13]:
for color in list(graph.keys())[:10]:
    print(shortest_path(graph, color, 'shiny gold'))

['muted tomato', 'faded gray', 'muted turquoise', 'drab silver', 'shiny gold']
None
None
None
['light aqua', 'muted tomato', 'faded gray', 'muted turquoise', 'drab silver', 'shiny gold']
None
None
['dark green', 'muted magenta', 'muted crimson', 'light bronze', 'bright yellow', 'shiny gold']
['faded bronze', 'muted gold', 'dark turquoise', 'dim aqua', 'muted maroon', 'vibrant red', 'pale chartreuse', 'shiny gold']
['clear lavender', 'dark beige', 'light green', 'shiny gold']


In [14]:
sum([shortest_path(graph, color, 'shiny gold') is not None for color in graph.keys()])

289

## Part 2

```
--- Part Two ---
It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

faded blue bags contain 0 other bags.
dotted black bags contain 0 other bags.
vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.
So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

```

In [48]:
def get_test_graph():
    s = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
    """
    return parse_graph(s)

test_graph = get_test_graph()
test_graph

{'shiny gold': {'dark red': 2},
 'dark red': {'dark orange': 2},
 'dark orange': {'dark yellow': 2},
 'dark yellow': {'dark green': 2},
 'dark green': {'dark blue': 2},
 'dark blue': {'dark violet': 2},
 'dark violet': {}}

In [95]:
def get_paths(graph, source, verbose=False):
    """Traverse the `graph` starting at `source` and return a list of tuples of ((node, count), path)
    where path is a list of (node, count) values.
    """
    paths = []
    queue = deque([(source, [(source, 1)])])
    while queue:
        if verbose:
            print(f'=== {queue} ===')
        vertex, path = queue.pop()
        if verbose:
            print(f'removed from queue: "{vertex}", {path}')
        for node_name, node_count in zip(graph.get(vertex).keys(), graph.get(vertex).values()):
            if verbose:
                print(f'\tnode_name: "{node_name}", node_count: {node_count}')
            new_path = path + [(node_name, node_count)]
            queue.append((node_name, new_path))
            if len(graph[node_name]) == 0:
                paths.append(new_path)
    return paths

get_paths(test_graph, 'shiny gold')

[[('shiny gold', 1),
  ('dark red', 2),
  ('dark orange', 2),
  ('dark yellow', 2),
  ('dark green', 2),
  ('dark blue', 2),
  ('dark violet', 2)]]

In [124]:
def calculate_path_cost(path, verbose=0):
    cost = {}
    previous_node_cost = None
    for node_name, node_count in path:
        if previous_node_cost is None:
            node_cost = node_count
        else:
            node_cost = previous_node_cost * node_count
        previous_node_cost = node_cost
        cost[node_name] = node_cost
        if verbose >= 2:
            print(node_name, node_count, cost)
    return cost


def calculate_paths_cost(paths, verbose=0):
    costs = {}
    for path in paths:
        #costs.append(calculate_path_cost(path, verbose=verbose))
        cost_dict = calculate_path_cost(path, verbose=verbose)
#         for k, v in cost_dict.items():
#             if k in costs:
#                 raise ValueError(f'key: "{k}", value: {v} is already in {costs}')
        costs.update(cost_dict)
        if verbose >= 1:
            print(f'>> {costs}')
    if verbose >= 1:
        print(f'>>> final costs: {costs}')
    root_node_cost = paths[0][0][1]
    return sum(costs.values()) - root_node_cost

calculate_paths_cost(get_paths(test_graph, 'shiny gold'), verbose=1)

>> {'shiny gold': 1, 'dark red': 2, 'dark orange': 4, 'dark yellow': 8, 'dark green': 16, 'dark blue': 32, 'dark violet': 64}
>>> final costs: {'shiny gold': 1, 'dark red': 2, 'dark orange': 4, 'dark yellow': 8, 'dark green': 16, 'dark blue': 32, 'dark violet': 64}


126

In [132]:
paths = get_paths(graph, 'shiny gold')
paths[1]

[('shiny gold', 1),
 ('wavy cyan', 3),
 ('clear indigo', 5),
 ('posh brown', 2),
 ('striped chartreuse', 1)]

In [133]:
calculate_path_cost(paths[1], verbose=2)

shiny gold 1 {'shiny gold': 1}
wavy cyan 3 {'shiny gold': 1, 'wavy cyan': 3}
clear indigo 5 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15}
posh brown 2 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30}
striped chartreuse 1 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30, 'striped chartreuse': 30}


{'shiny gold': 1,
 'wavy cyan': 3,
 'clear indigo': 15,
 'posh brown': 30,
 'striped chartreuse': 30}

In [134]:
calculate_paths_cost(paths[0:2], verbose=2)

shiny gold 1 {'shiny gold': 1}
wavy cyan 3 {'shiny gold': 1, 'wavy cyan': 3}
clear indigo 5 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15}
posh brown 2 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30}
mirrored crimson 5 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30, 'mirrored crimson': 150}
>> {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30, 'mirrored crimson': 150}
shiny gold 1 {'shiny gold': 1}
wavy cyan 3 {'shiny gold': 1, 'wavy cyan': 3}
clear indigo 5 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15}
posh brown 2 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30}
striped chartreuse 1 {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30, 'striped chartreuse': 30}
>> {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 30, 'mirrored crimson': 150, 'striped chartreuse': 30}
>>> final costs: {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh 

228

In [128]:
# {'shiny gold': 1, 'wavy cyan': 3, 'clear indigo': 15, 'posh brown': 8, 'mirrored crimson': 16, 'striped chartreuse': 8, 'dark lime': 8, 'dull chartreuse': 24, 'dim violet': 40, 'dull white': 120, 'mirrored olive': 45, 'faded beige': 16, 'bright turquoise': 6, 'vibrant bronze': 6, 'dotted gold': 24, 'drab beige': 24, 'clear yellow': 48, 'posh plum': 40, 'dim lime': 9, 'faded silver': 15, 'mirrored brown': 9, 'mirrored chartreuse': 9, 'pale magenta': 2, 'dull tomato': 10, 'dim plum': 8, 'drab salmon': 16, 'light coral': 64, 'dotted orange': 32, 'posh white': 5, 'pale gray': 10, 'bright blue': 25, 'shiny coral': 1, 'muted bronze': 4, 'mirrored beige': 8, 'striped olive': 8}

In [129]:
# calculate_paths_cost(get_paths(graph, 'shiny gold'), verbose=1)

In [149]:
class Bag():
    def __init__(self, graph, name, count):
        self.name = name
        self.count = count
        self.children = []
        for child_name, child_count in graph[name].items():
            self.children.append(Bag(graph, child_name, child_count))
    
    def __repr__(self):
        return self.to_string(level=0)
    
    def to_string(self, level=0):
        tabs = '.' * level
        s = [f'{tabs}name: {self.name}, count: {self.count}']
        for child in self.children:
            s.append(child.to_string(level=level + 1))
        return '\n'.join(s)
    
    def total_count(self):
        return self.count + self.count * sum([child.total_count() for child in self.children])

test_bag = Bag(test_graph, 'shiny gold', 1)
test_bag

name: shiny gold, count: 1
.name: dark red, count: 2
..name: dark orange, count: 2
...name: dark yellow, count: 2
....name: dark green, count: 2
.....name: dark blue, count: 2
......name: dark violet, count: 2

In [150]:
test_bag.total_count()

127

In [154]:
bag = Bag(graph, 'shiny gold', 1)
bag

name: shiny gold, count: 1
.name: shiny coral, count: 1
..name: pale magenta, count: 2
...name: dim plum, count: 4
....name: posh brown, count: 1
.....name: dark lime, count: 1
......name: dull chartreuse, count: 3
......name: dim violet, count: 5
.......name: dull white, count: 3
......name: mirrored crimson, count: 2
.....name: mirrored crimson, count: 5
.....name: striped chartreuse, count: 1
....name: dotted orange, count: 4
.....name: light coral, count: 2
....name: dotted gold, count: 3
.....name: drab beige, count: 1
.....name: mirrored crimson, count: 1
....name: drab salmon, count: 2
.....name: light coral, count: 5
...name: dull tomato, count: 5
....name: posh plum, count: 4
...name: bright turquoise, count: 3
....name: dotted gold, count: 2
.....name: drab beige, count: 1
.....name: mirrored crimson, count: 1
....name: dim violet, count: 2
.....name: dull white, count: 3
....name: vibrant bronze, count: 1
...name: dark lime, count: 1
....name: dull chartreuse, count: 3
....n

In [156]:
bag.total_count() - 1

30055